In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

Hyperparameter tuning with Ray Tune
===================================

Hyperparameter tuning can make the difference between an average model
and a highly accurate one. Often simple things like choosing a different
learning rate or changing a network layer size can have a dramatic
impact on your model performance.

Fortunately, there are tools that help with finding the best combination
of parameters. [Ray Tune](https://docs.ray.io/en/latest/tune.html) is an
industry standard tool for distributed hyperparameter tuning. Ray Tune
includes the latest hyperparameter search algorithms, integrates with
TensorBoard and other analysis libraries, and natively supports
distributed training through [Ray\'s distributed machine learning
engine](https://ray.io/).

In this tutorial, we will show you how to integrate Ray Tune into your
PyTorch training workflow. We will extend [this tutorial from the
PyTorch
documentation](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)
for training a CIFAR10 image classifier.

As you will see, we only need to add some slight modifications. In
particular, we need to

1.  wrap data loading and training in functions,
2.  make some network parameters configurable,
3.  add checkpointing (optional),
4.  and define the search space for the model tuning

| 

To run this tutorial, please make sure the following packages are
installed:

-   `ray[tune]`: Distributed hyperparameter tuning library
-   `torchvision`: For the data transformers

Setup / Imports
---------------

Let\'s start with the imports:


In [2]:
from functools import partial
import os
import tempfile
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray import train
from ray.train import Checkpoint, get_checkpoint
from ray.tune.schedulers import ASHAScheduler
import ray.cloudpickle as pickle

Most of the imports are needed for building the PyTorch model. Only the
last imports are for Ray Tune.

Data loaders
============

We wrap the data loaders in their own function and pass a global data
directory. This way we can share a data directory between different
trials.


In [3]:
def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )

    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform
    )

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform
    )

    return trainset, testset

Configurable neural network
===========================

We can only tune those parameters that are configurable. In this
example, we can specify the layer sizes of the fully connected layers:


In [4]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

The train function
==================

Now it gets interesting, because we introduce some changes to the
example [from the PyTorch
documentation](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).

We wrap the training script in a function
`train_cifar(config, data_dir=None)`. The `config` parameter will
receive the hyperparameters we would like to train with. The `data_dir`
specifies the directory where we load and store the data, so that
multiple runs can share the same data source. We also load the model and
optimizer state at the start of the run, if a checkpoint is provided.
Further down in this tutorial you will find information on how to save
the checkpoint and what it is used for.

``` {.sourceCode .python}
net = Net(config["l1"], config["l2"])

checkpoint = get_checkpoint()
if checkpoint:
    with checkpoint.as_directory() as checkpoint_dir:
        data_path = Path(checkpoint_dir) / "data.pkl"
        with open(data_path, "rb") as fp:
            checkpoint_state = pickle.load(fp)
        start_epoch = checkpoint_state["epoch"]
        net.load_state_dict(checkpoint_state["net_state_dict"])
        optimizer.load_state_dict(checkpoint_state["optimizer_state_dict"])
else:
    start_epoch = 0
```

The learning rate of the optimizer is made configurable, too:

``` {.sourceCode .python}
optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
```

We also split the training data into a training and validation subset.
We thus train on 80% of the data and calculate the validation loss on
the remaining 20%. The batch sizes with which we iterate through the
training and test sets are configurable as well.

Adding (multi) GPU support with DataParallel
--------------------------------------------

Image classification benefits largely from GPUs. Luckily, we can
continue to use PyTorch\'s abstractions in Ray Tune. Thus, we can wrap
our model in `nn.DataParallel` to support data parallel training on
multiple GPUs:

``` {.sourceCode .python}
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
    if torch.cuda.device_count() > 1:
        net = nn.DataParallel(net)
net.to(device)
```

By using a `device` variable we make sure that training also works when
we have no GPUs available. PyTorch requires us to send our data to the
GPU memory explicitly, like this:

``` {.sourceCode .python}
for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
```

The code now supports training on CPUs, on a single GPU, and on multiple
GPUs. Notably, Ray also supports [fractional
GPUs](https://docs.ray.io/en/master/using-ray-with-gpus.html#fractional-gpus)
so we can share GPUs among trials, as long as the model still fits on
the GPU memory. We\'ll come back to that later.

Communicating with Ray Tune
---------------------------

The most interesting part is the communication with Ray Tune:

``` {.sourceCode .python}
checkpoint_data = {
    "epoch": epoch,
    "net_state_dict": net.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
}
with tempfile.TemporaryDirectory() as checkpoint_dir:
    data_path = Path(checkpoint_dir) / "data.pkl"
    with open(data_path, "wb") as fp:
        pickle.dump(checkpoint_data, fp)

    checkpoint = Checkpoint.from_directory(checkpoint_dir)
    train.report(
        {"loss": val_loss / val_steps, "accuracy": correct / total},
        checkpoint=checkpoint,
    )
```

Here we first save a checkpoint and then report some metrics back to Ray
Tune. Specifically, we send the validation loss and accuracy back to Ray
Tune. Ray Tune can then use these metrics to decide which hyperparameter
configuration lead to the best results. These metrics can also be used
to stop bad performing trials early in order to avoid wasting resources
on those trials.

The checkpoint saving is optional, however, it is necessary if we wanted
to use advanced schedulers like [Population Based
Training](https://docs.ray.io/en/latest/tune/examples/pbt_guide.html).
Also, by saving the checkpoint we can later load the trained models and
validate them on a test set. Lastly, saving checkpoints is useful for
fault tolerance, and it allows us to interrupt training and continue
training later.

Full training function
----------------------

The full code example looks like this:


In [5]:
def train_cifar(config, data_dir=None):
    net = Net(config["l1"], config["l2"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    checkpoint = get_checkpoint()
    if checkpoint:
        with checkpoint.as_directory() as checkpoint_dir:
            data_path = Path(checkpoint_dir) / "data.pkl"
            with open(data_path, "rb") as fp:
                checkpoint_state = pickle.load(fp)
            start_epoch = checkpoint_state["epoch"]
            net.load_state_dict(checkpoint_state["net_state_dict"])
            optimizer.load_state_dict(checkpoint_state["optimizer_state_dict"])
    else:
        start_epoch = 0

    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs]
    )

    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=8
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=int(config["batch_size"]), shuffle=True, num_workers=8
    )

    for epoch in range(start_epoch, 10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print(
                    "[%d, %5d] loss: %.3f"
                    % (epoch + 1, i + 1, running_loss / epoch_steps)
                )
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        checkpoint_data = {
            "epoch": epoch,
            "net_state_dict": net.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        }
        with tempfile.TemporaryDirectory() as checkpoint_dir:
            data_path = Path(checkpoint_dir) / "data.pkl"
            with open(data_path, "wb") as fp:
                pickle.dump(checkpoint_data, fp)

            checkpoint = Checkpoint.from_directory(checkpoint_dir)
            train.report(
                {"loss": val_loss / val_steps, "accuracy": correct / total},
                checkpoint=checkpoint,
            )
    
    print("Finished Training")

As you can see, most of the code is adapted directly from the original
example.

Test set accuracy
=================

Commonly the performance of a machine learning model is tested on a
hold-out test set with data that has not been used for training the
model. We also wrap this in a function:


In [6]:
def test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

The function also expects a `device` parameter, so we can do the test
set validation on a GPU.

Configuring the search space
============================

Lastly, we need to define Ray Tune\'s search space. Here is an example:

``` {.sourceCode .python}
config = {
    "l1": tune.choice([2 ** i for i in range(9)]),
    "l2": tune.choice([2 ** i for i in range(9)]),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}
```

The `tune.choice()` accepts a list of values that are uniformly sampled
from. In this example, the `l1` and `l2` parameters should be powers of
2 between 4 and 256, so either 4, 8, 16, 32, 64, 128, or 256. The `lr`
(learning rate) should be uniformly sampled between 0.0001 and 0.1.
Lastly, the batch size is a choice between 2, 4, 8, and 16.

At each trial, Ray Tune will now randomly sample a combination of
parameters from these search spaces. It will then train a number of
models in parallel and find the best performing one among these. We also
use the `ASHAScheduler` which will terminate bad performing trials
early.

We wrap the `train_cifar` function with `functools.partial` to set the
constant `data_dir` parameter. We can also tell Ray Tune what resources
should be available for each trial:

``` {.sourceCode .python}
gpus_per_trial = 2
# ...
result = tune.run(
    partial(train_cifar, data_dir=data_dir),
    resources_per_trial={"cpu": 8, "gpu": gpus_per_trial},
    config=config,
    num_samples=num_samples,
    scheduler=scheduler,
    checkpoint_at_end=True)
```

You can specify the number of CPUs, which are then available e.g. to
increase the `num_workers` of the PyTorch `DataLoader` instances. The
selected number of GPUs are made visible to PyTorch in each trial.
Trials do not have access to GPUs that haven\'t been requested for them
- so you don\'t have to care about two trials using the same set of
resources.

Here we can also specify fractional GPUs, so something like
`gpus_per_trial=0.5` is completely valid. The trials will then share
GPUs among each other. You just have to make sure that the models still
fit in the GPU memory.

After training the models, we will find the best performing one and load
the trained network from the checkpoint file. We then obtain the test
set accuracy and report everything by printing.

The full main function looks like this:


In [7]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    data_dir = os.path.abspath("./data")
    load_data(data_dir)
    config = {
        "l1": tune.choice([2**i for i in range(9)]),
        "l2": tune.choice([2**i for i in range(9)]),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16]),
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2,
    )
    result = tune.run(
        partial(train_cifar, data_dir=data_dir),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
    )

    best_trial = result.get_best_trial("loss", "min", "last")
    print(f"Best trial config: {best_trial.config}")
    print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
    print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")

    best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint = result.get_best_checkpoint(trial=best_trial, metric="accuracy", mode="max")
    with best_checkpoint.as_directory() as checkpoint_dir:
        data_path = Path(checkpoint_dir) / "data.pkl"
        with open(data_path, "rb") as fp:
            best_checkpoint_data = pickle.load(fp)

        best_trained_model.load_state_dict(best_checkpoint_data["net_state_dict"])
        test_acc = test_accuracy(best_trained_model, device)
        print("Best trial test set accuracy: {}".format(test_acc))


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=10, max_num_epochs=10, gpus_per_trial=0)

Files already downloaded and verified
Files already downloaded and verified


2024-06-05 17:10:01,496	INFO worker.py:1749 -- Started a local Ray instance.
2024-06-05 17:10:02,616	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2024-06-05 17:10:02,620	INFO tune.py:614 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(func pid=227895) Files already downloaded and verified
(func pid=227851) [1,  2000] loss: 2.344
(func pid=227894) Files already downloaded and verified [repeated 19x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


Trial name,accuracy,loss,should_checkpoint
train_cifar_97c9d_00000,0.2052,1.89251,True
train_cifar_97c9d_00001,0.1011,2.33847,True
train_cifar_97c9d_00002,0.0985,2.30294,True
train_cifar_97c9d_00003,0.1588,2.19483,True
train_cifar_97c9d_00004,0.134,2.29062,True
train_cifar_97c9d_00005,0.0977,2.30723,True
train_cifar_97c9d_00006,0.5176,1.41599,True
train_cifar_97c9d_00007,0.4821,1.42622,True
train_cifar_97c9d_00008,0.5535,1.30257,True
train_cifar_97c9d_00009,0.3863,1.69042,True


(func pid=227895) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/luiz.luz/ray_results/train_cifar_2024-06-05_17-10-02/train_cifar_97c9d_00007_7_batch_size=16,l1=8,l2=64,lr=0.0002_2024-06-05_17-10-02/checkpoint_000000)


...

(func pid=227894) [7,  2000] loss: 1.277
(func pid=227894) [7,  4000] loss: 0.637
(func pid=227894) [7,  6000] loss: 0.434
(func pid=227894) [7,  8000] loss: 0.329
(func pid=227894) [7, 10000] loss: 0.262


(func pid=227894) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/luiz.luz/ray_results/train_cifar_2024-06-05_17-10-02/train_cifar_97c9d_00006_6_batch_size=4,l1=64,l2=8,lr=0.0027_2024-06-05_17-10-02/checkpoint_000006)


(func pid=227894) [8,  2000] loss: 1.242
(func pid=227894) [8,  4000] loss: 0.643
(func pid=227894) [8,  6000] loss: 0.428
(func pid=227894) [8,  8000] loss: 0.325
(func pid=227894) [8, 10000] loss: 0.256


2024-06-05 17:15:32,949	INFO tune.py:1007 -- Wrote the latest version of all result files and experiment state to '/home/luiz.luz/ray_results/train_cifar_2024-06-05_17-10-02' in 0.0065s.
2024-06-05 17:15:32,957	INFO tune.py:1039 -- Total run time: 330.34 seconds (330.28 seconds for the tuning loop).
(func pid=227894) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/luiz.luz/ray_results/train_cifar_2024-06-05_17-10-02/train_cifar_97c9d_00006_6_batch_size=4,l1=64,l2=8,lr=0.0027_2024-06-05_17-10-02/checkpoint_000007)


Best trial config: {'l1': 16, 'l2': 256, 'lr': 0.0072942022748702755, 'batch_size': 16}
Best trial final validation loss: 1.3025706018447876
Best trial final validation accuracy: 0.5535
Files already downloaded and verified
Files already downloaded and verified
Best trial test set accuracy: 0.5637


If you run the code, an example output could look like this:

``` {.sourceCode .sh}
Number of trials: 10/10 (10 TERMINATED)
+-----+--------------+------+------+-------------+--------+---------+------------+
| ... |   batch_size |   l1 |   l2 |          lr |   iter |    loss |   accuracy |
|-----+--------------+------+------+-------------+--------+---------+------------|
| ... |            2 |    1 |  256 | 0.000668163 |      1 | 2.31479 |     0.0977 |
| ... |            4 |   64 |    8 | 0.0331514   |      1 | 2.31605 |     0.0983 |
| ... |            4 |    2 |    1 | 0.000150295 |      1 | 2.30755 |     0.1023 |
| ... |           16 |   32 |   32 | 0.0128248   |     10 | 1.66912 |     0.4391 |
| ... |            4 |    8 |  128 | 0.00464561  |      2 | 1.7316  |     0.3463 |
| ... |            8 |  256 |    8 | 0.00031556  |      1 | 2.19409 |     0.1736 |
| ... |            4 |   16 |  256 | 0.00574329  |      2 | 1.85679 |     0.3368 |
| ... |            8 |    2 |    2 | 0.00325652  |      1 | 2.30272 |     0.0984 |
| ... |            2 |    2 |    2 | 0.000342987 |      2 | 1.76044 |     0.292  |
| ... |            4 |   64 |   32 | 0.003734    |      8 | 1.53101 |     0.4761 |
+-----+--------------+------+------+-------------+--------+---------+------------+

Best trial config: {'l1': 64, 'l2': 32, 'lr': 0.0037339984519545164, 'batch_size': 4}
Best trial final validation loss: 1.5310075663924216
Best trial final validation accuracy: 0.4761
Best trial test set accuracy: 0.4737
```

Most trials have been stopped early in order to avoid wasting resources.
The best performing trial achieved a validation accuracy of about 47%,
which could be confirmed on the test set.

So that\'s it! You can now tune the parameters of your PyTorch models.
